In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Cargar datos
df = pd.read_csv('data/alquiler_AMBA_dev.csv')

print(f"Dataset original: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")

Dataset original: (278725, 45)
Columnas: ['id_grid', 'MesListing', 'TIPOPROPIEDAD', 'STotalM2', 'SConstrM2', 'Dormitorios', 'Banos', 'Ambientes', 'SitioOrigen', 'Amoblado', 'Antiguedad', 'Cisterna', 'AccesoInternet', 'BusinessCenter', 'Gimnasio', 'Laundry', 'Calefaccion', 'SalonDeUsosMul', 'AireAC', 'Recepcion', 'Estacionamiento', 'Jacuzzi', 'AreaJuegosInfantiles', 'Chimenea', 'Ascensor', 'SalonFiestas', 'Seguridad', 'Pileta', 'Cocheras', 'PistaJogging', 'EstacionamientoVisitas', 'Lobby', 'LocalesComerciales', 'SistContraIncendios', 'AreaParrillas', 'CanchaTennis', 'AreaCine', 'ITE_ADD_CITY_NAME', 'ITE_ADD_STATE_NAME', 'ITE_ADD_NEIGHBORHOOD_NAME', 'ITE_TIPO_PROD', 'LONGITUDE', 'LATITUDE', 'precio_pesos_constantes', 'year']


In [9]:
# Guardar shape inicial
initial_rows = len(df)

# 1.1 Eliminar duplicados exactos
df_clean = df.drop_duplicates()
print(f"Duplicados exactos eliminados: {initial_rows - len(df_clean)}")

# 1.2 Identificar y eliminar re-listings (mismo inmueble en múltiples meses)
# Ordenar por fecha para mantener el más reciente
df_clean['MesListing'] = pd.to_datetime(df_clean['MesListing'])
df_clean = df_clean.sort_values('MesListing', ascending=False)

# Definir columnas clave para identificar mismo inmueble
key_cols = ['LONGITUDE', 'LATITUDE', 'STotalM2', 'SConstrM2', 
            'Dormitorios', 'Banos', 'Ambientes']

# Mantener solo la publicación más reciente por inmueble
df_clean = df_clean.drop_duplicates(subset=key_cols, keep='first')
print(f"Re-listings eliminados: {initial_rows - len(df_clean)}")

# 1.3 Filtrar registros con datos críticos faltantes
critical_cols = ['precio_pesos_constantes', 'LONGITUDE', 'LATITUDE']
df_clean = df_clean.dropna(subset=critical_cols)
print(f"Registros con datos críticos faltantes eliminados: {initial_rows - len(df_clean)}")

print(f"\nDataset después de deduplicación: {df_clean.shape[0]} filas")
print(f"Reducción total: {100 * (1 - len(df_clean)/initial_rows):.2f}%")

Duplicados exactos eliminados: 7040
Re-listings eliminados: 172461
Registros con datos críticos faltantes eliminados: 172461

Dataset después de deduplicación: 106264 filas
Reducción total: 61.87%


In [10]:
# 2.1 Calcular price_per_m2 para identificar ventas
df_clean['price_per_m2_temp'] = df_clean['precio_pesos_constantes'] / df_clean['STotalM2']

# 2.2 Filtrar por precio absoluto (umbral conservador para alquileres)
rent_filter = df_clean['precio_pesos_constantes'] <= 500000
df_rent = df_clean[rent_filter].copy()
print(f"Propiedades eliminadas por precio >$500k: {(~rent_filter).sum()}")

# 2.3 Filtrar por precio por m² (eliminar posibles ventas)
price_m2_filter = df_rent['price_per_m2_temp'] <= 1000
df_rent = df_rent[price_m2_filter].copy()
print(f"Propiedades eliminadas por price_per_m2 >$1000: {(~price_m2_filter).sum()}")

# 2.4 Eliminar columna temporal
df_rent = df_rent.drop(columns=['price_per_m2_temp'])

print(f"\nDataset enfocado en alquileres: {df_rent.shape[0]} filas")
print(f"Reducción por filtrado de ventas: {100 * (1 - len(df_rent)/len(df_clean)):.2f}%")

# Actualizar df principal
df = df_rent.copy()
del df_clean, df_rent

Propiedades eliminadas por precio >$500k: 764
Propiedades eliminadas por price_per_m2 >$1000: 4002

Dataset enfocado en alquileres: 101498 filas
Reducción por filtrado de ventas: 4.49%


In [11]:
# 3.1 Filtrar precios anómalos
price_filter = (df['precio_pesos_constantes'] >= 1000) & \
               (df['precio_pesos_constantes'] <= 200000)
df = df[price_filter].copy()
print(f"Outliers de precio eliminados: {(~price_filter).sum()}")

# 3.2 Filtrar superficies anómalas
size_filter = (df['STotalM2'] > 0) & (df['STotalM2'] <= 1000)
df = df[size_filter].copy()
print(f"Outliers de STotalM2 eliminados: {(~size_filter).sum()}")

# 3.3 Filtrar SConstrM2 anómalas y validar lógica
sconstr_filter = (df['SConstrM2'] > 0) & \
                 (df['SConstrM2'] <= 1000) & \
                 (df['SConstrM2'] <= df['STotalM2'])
df = df[sconstr_filter].copy()
print(f"Outliers de SConstrM2 eliminados: {(~sconstr_filter).sum()}")

# 3.4 Filtrar dormitorios anómalos
dorm_filter = (df['Dormitorios'] >= 0) & (df['Dormitorios'] <= 8)
df = df[dorm_filter].copy()
print(f"Outliers de Dormitorios eliminados: {(~dorm_filter).sum()}")

# 3.5 Filtrar baños anómalos
banos_filter = (df['Banos'] >= 0) & (df['Banos'] <= 8)
df = df[banos_filter].copy()
print(f"Outliers de Banos eliminados: {(~banos_filter).sum()}")

# 3.6 Filtrar ambientes anómalos
ambientes_filter = (df['Ambientes'] >= 0) & (df['Ambientes'] <= 10)
df = df[ambientes_filter].copy()
print(f"Outliers de Ambientes eliminados: {(~ambientes_filter).sum()}")

# 3.7 Filtrar cocheras anómalas
cocheras_filter = (df['Cocheras'] >= 0) & (df['Cocheras'] <= 10)
df = df[cocheras_filter].copy()
print(f"Outliers de Cocheras eliminados: {(~cocheras_filter).sum()}")

# 3.8 Validar consistencia dormitorios <= ambientes
consistency_filter = df['Dormitorios'] <= df['Ambientes']
df = df[consistency_filter].copy()
print(f"Registros inconsistentes (Dorm > Amb) eliminados: {(~consistency_filter).sum()}")

print(f"\nDataset después de limpieza de outliers: {df.shape[0]} filas")

Outliers de precio eliminados: 399
Outliers de STotalM2 eliminados: 167
Outliers de SConstrM2 eliminados: 2635
Outliers de Dormitorios eliminados: 46
Outliers de Banos eliminados: 28
Outliers de Ambientes eliminados: 28
Outliers de Cocheras eliminados: 34
Registros inconsistentes (Dorm > Amb) eliminados: 301

Dataset después de limpieza de outliers: 97860 filas


In [12]:
# 4.1 Columnas sin varianza
cols_to_drop = ['TIPOPROPIEDAD']  # Solo "Departamento"

# 4.2 Columnas con >25% faltantes y baja relevancia
high_missing_cols = [
    'SitioOrigen',
    'PistaJogging',
    'Lobby', 
    'LocalesComerciales',
    'BusinessCenter',
    'Chimenea',
    'CanchaTennis',
    'SalonFiestas',
    'Estacionamiento',
    'EstacionamientoVisitas',
    'Recepcion',
    'Cisterna',
    'AreaJuegosInfantiles',
    'AreaCine',
    'SistContraIncendios'
]

cols_to_drop.extend(high_missing_cols)

# 4.3 Eliminar SConstrM2 por multicolinealidad perfecta con STotalM2
cols_to_drop.append('SConstrM2')

# 4.4 Eliminar columnas
df = df.drop(columns=cols_to_drop)

print(f"Columnas eliminadas: {len(cols_to_drop)}")
print(f"Columnas restantes: {df.shape[1]}")
print(f"\nColumnas eliminadas:")
for col in cols_to_drop:
    print(f"  - {col}")

Columnas eliminadas: 17
Columnas restantes: 28

Columnas eliminadas:
  - TIPOPROPIEDAD
  - SitioOrigen
  - PistaJogging
  - Lobby
  - LocalesComerciales
  - BusinessCenter
  - Chimenea
  - CanchaTennis
  - SalonFiestas
  - Estacionamiento
  - EstacionamientoVisitas
  - Recepcion
  - Cisterna
  - AreaJuegosInfantiles
  - AreaCine
  - SistContraIncendios
  - SConstrM2


In [13]:
def clean_antiguedad(value):
    """Extrae años numéricos de formato mixto"""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip()
    
    # Casos especiales
    if value_str in ['0', '0.0', '0 años']:
        return 0
    
    # Extraer número
    import re
    match = re.search(r'(\d+)', value_str)
    if match:
        years = int(match.group(1))
        # Filtrar valores absurdos (>200 años)
        return years if years <= 200 else np.nan
    
    return np.nan

# Aplicar limpieza
df['Antiguedad'] = df['Antiguedad'].apply(clean_antiguedad)

print(f"Antiguedad limpiada. Valores únicos: {df['Antiguedad'].nunique()}")
print(f"Rango: [{df['Antiguedad'].min()}, {df['Antiguedad'].max()}]")
print(f"Valores nulos: {df['Antiguedad'].isna().sum()} ({100*df['Antiguedad'].isna().sum()/len(df):.2f}%)")

Antiguedad limpiada. Valores únicos: 119
Rango: [0.0, 200.0]
Valores nulos: 11429 (11.68%)


In [14]:
def standardize_binary(value):
    """Convierte valores mixtos a binario 0/1"""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().lower()
    
    # Valores positivos
    if value_str in ['sí', 'si', '1', '1.0']:
        return 1
    
    # Valores negativos
    if value_str in ['no', '0', '0.0', ' 0']:
        return 0
    
    return np.nan

# Lista de columnas de amenities a estandarizar
amenity_cols = [
    'Amoblado',
    'AccesoInternet',
    'Gimnasio',
    'Laundry',
    'Calefaccion',
    'SalonDeUsosMul',
    'AireAC',
    'Jacuzzi',
    'Ascensor',
    'Seguridad',
    'Pileta'
]

# Aplicar estandarización
for col in amenity_cols:
    if col in df.columns:
        df[col] = df[col].apply(standardize_binary)
        print(f"{col}: {df[col].value_counts().to_dict()}")

print(f"\nAmenities estandarizadas: {len([c for c in amenity_cols if c in df.columns])}")

Amoblado: {0.0: 78548, 1.0: 4909}
AccesoInternet: {0.0: 67924, 1.0: 15893}
Gimnasio: {0.0: 75697, 1.0: 6815}
Laundry: {0.0: 63992, 1.0: 20411}
Calefaccion: {0.0: 69385, 1.0: 14444}
SalonDeUsosMul: {0.0: 71320, 1.0: 10685}
AireAC: {0.0: 59607, 1.0: 25491}
Jacuzzi: {0.0: 79435, 1.0: 2074}
Ascensor: {0.0: 50303, 1.0: 10897}
Seguridad: {0.0: 74616, 1.0: 7678}
Pileta: {0.0: 72071, 1.0: 12551}

Amenities estandarizadas: 11


In [15]:
# 7.1 Imputar amenities con 0 (ausencia de amenity)
amenity_cols_in_df = [col for col in amenity_cols if col in df.columns]
df[amenity_cols_in_df] = df[amenity_cols_in_df].fillna(0)

print("Amenities imputadas con 0:")
for col in amenity_cols_in_df:
    print(f"  - {col}: {df[col].isna().sum()} nulos restantes")

# 7.2 Imputar AreaParrillas con 0
if 'AreaParrillas' in df.columns:
    df['AreaParrillas'] = df['AreaParrillas'].fillna(0)
    print(f"AreaParrillas imputada con 0")

# 7.3 Imputar STotalM2 con mediana (muy pocos faltantes)
if df['STotalM2'].isna().sum() > 0:
    median_stotal = df['STotalM2'].median()
    df['STotalM2'] = df['STotalM2'].fillna(median_stotal)
    print(f"STotalM2 imputada con mediana: {median_stotal}")

# 7.4 Imputar Antiguedad con mediana
if df['Antiguedad'].isna().sum() > 0:
    median_antiguedad = df['Antiguedad'].median()
    df['Antiguedad'] = df['Antiguedad'].fillna(median_antiguedad)
    print(f"Antiguedad imputada con mediana: {median_antiguedad}")

# 7.5 Imputar ITE_ADD_NEIGHBORHOOD_NAME con 'Unknown'
if df['ITE_ADD_NEIGHBORHOOD_NAME'].isna().sum() > 0:
    df['ITE_ADD_NEIGHBORHOOD_NAME'] = df['ITE_ADD_NEIGHBORHOOD_NAME'].fillna('Unknown')
    print(f"ITE_ADD_NEIGHBORHOOD_NAME imputada con 'Unknown'")

print(f"\nValores nulos restantes por columna:")
print(df.isna().sum()[df.isna().sum() > 0])

Amenities imputadas con 0:
  - Amoblado: 0 nulos restantes
  - AccesoInternet: 0 nulos restantes
  - Gimnasio: 0 nulos restantes
  - Laundry: 0 nulos restantes
  - Calefaccion: 0 nulos restantes
  - SalonDeUsosMul: 0 nulos restantes
  - AireAC: 0 nulos restantes
  - Jacuzzi: 0 nulos restantes
  - Ascensor: 0 nulos restantes
  - Seguridad: 0 nulos restantes
  - Pileta: 0 nulos restantes
AreaParrillas imputada con 0
Antiguedad imputada con mediana: 8.0
ITE_ADD_NEIGHBORHOOD_NAME imputada con 'Unknown'

Valores nulos restantes por columna:
Series([], dtype: int64)


In [16]:
# 8.1 Precio por m²
df['precio_por_m2'] = df['precio_pesos_constantes'] / df['STotalM2']

# 8.2 Densidad de ambientes
df['densidad_ambientes'] = df['Ambientes'] / df['STotalM2']

# 8.3 Total de amenities
amenity_binary_cols = [col for col in amenity_cols_in_df if col in df.columns]
if 'AreaParrillas' in df.columns:
    amenity_binary_cols.append('AreaParrillas')

df['total_amenities'] = df[amenity_binary_cols].sum(axis=1)

# 8.4 Tiene cochera
df['tiene_cochera'] = (df['Cocheras'] > 0).astype(int)

# 8.5 Features temporales
df['mes'] = df['MesListing'].dt.month
df['year'] = df['year'].astype(int)

# 8.6 Estacionalidad
def get_season(month):
    if month in [12, 1, 2]:
        return 'verano'
    elif month in [3, 4, 5]:
        return 'otoño'
    elif month in [6, 7, 8]:
        return 'invierno'
    else:
        return 'primavera'

df['estacion'] = df['mes'].apply(get_season)

print("Features creadas:")
print("  - precio_por_m2")
print("  - densidad_ambientes")
print("  - total_amenities")
print("  - tiene_cochera")
print("  - mes, year, estacion")
print(f"\nShape final: {df.shape}")

Features creadas:
  - precio_por_m2
  - densidad_ambientes
  - total_amenities
  - tiene_cochera
  - mes, year, estacion

Shape final: (97860, 34)


In [17]:
# 9.1 Verificar valores nulos
print("=== VERIFICACIÓN FINAL ===")
print(f"\nShape final: {df.shape}")
print(f"\nValores nulos por columna:")
null_counts = df.isna().sum()
if null_counts.sum() > 0:
    print(null_counts[null_counts > 0])
else:
    print("No hay valores nulos")

# 9.2 Verificar tipos de datos
print(f"\nTipos de datos:")
print(df.dtypes.value_counts())

# 9.3 Estadísticas del target
print(f"\n=== ESTADÍSTICAS DEL TARGET ===")
print(f"Precio (pesos constantes):")
print(f"  Count: {df['precio_pesos_constantes'].count():,}")
print(f"  Mean: ${df['precio_pesos_constantes'].mean():,.2f}")
print(f"  Median: ${df['precio_pesos_constantes'].median():,.2f}")
print(f"  Std: ${df['precio_pesos_constantes'].std():,.2f}")
print(f"  Min: ${df['precio_pesos_constantes'].min():,.2f}")
print(f"  Max: ${df['precio_pesos_constantes'].max():,.2f}")

# 9.4 Guardar dataset limpio
df.to_csv('alquiler_AMBA_clean.csv', index=False)
print(f"\n✓ Dataset limpio guardado: alquiler_AMBA_clean.csv")

=== VERIFICACIÓN FINAL ===

Shape final: (97860, 34)

Valores nulos por columna:
No hay valores nulos

Tipos de datos:
float64           20
int64              7
object             5
datetime64[ns]     1
int32              1
Name: count, dtype: int64

=== ESTADÍSTICAS DEL TARGET ===
Precio (pesos constantes):
  Count: 97,860
  Mean: $14,682.00
  Median: $8,408.60
  Std: $20,794.63
  Min: $1,010.40
  Max: $199,695.79

✓ Dataset limpio guardado: alquiler_AMBA_clean.csv


In [ ]:
# Resumen final
print("="*80)
print("RESUMEN DE TRANSFORMACIONES APLICADAS")
print("="*80)
print(f"""
1. Deduplicación y filtrado básico
2. Filtrado de propiedades en venta (>$500k o price_per_m2 >$1000)
3. Eliminación de outliers extremos en todas las variables numéricas
4. Eliminación de {len(cols_to_drop)} columnas irrelevantes
5. Limpieza y extracción numérica de Antiguedad
6. Estandarización de {len(amenity_cols_in_df)} amenities binarias a 0/1
7. Imputación de valores faltantes
8. Creación de 7 features derivadas
9. Validación de consistencia lógica

Dataset final: {df.shape[0]} filas x {df.shape[1]} columnas
Listo para modelado
""")

RESUMEN DE TRANSFORMACIONES APLICADAS

1. Deduplicación y filtrado básico
2. Filtrado de propiedades en venta (>$500k o price_per_m2 >$1000)
3. Eliminación de outliers extremos en todas las variables numéricas
4. Eliminación de 17 columnas irrelevantes
5. Limpieza y extracción numérica de Antiguedad
6. Estandarización de 11 amenities binarias a 0/1
7. Imputación de valores faltantes
8. Creación de 7 features derivadas
9. Validación de consistencia lógica

Dataset final: 97860 filas × 34 columnas
Listo para modelado

